In [1]:
try:
    import pickle
    import data_loader as dl
    import nltk
    import torch
    import numpy as np
    from torch import tensor
    from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
    from torch.nn import CrossEntropyLoss
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.utils.class_weight import compute_class_weight
    from torch.utils.data import Dataset, DataLoader
    from tqdm import tqdm
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import classification_report, confusion_matrix
    import os
    from transformers import BertTokenizerFast
    from sklearn.metrics import f1_score
    from collections import Counter
    import matplotlib.pyplot as plt
    import seaborn as sns
    import random
    from torch.nn.utils import clip_grad_norm_
    

    print("All imports successful!")

except ImportError as e:
    print(f"ImportError: {e}")

All imports successful!


In [2]:
def load_model(path, model_class=BertForSequenceClassification, num_labels=10):
    with open(path, 'rb') as f:
        checkpoint = pickle.load(f)
    
    # Load the model
    model = model_class.from_pretrained('bert-base-uncased', num_labels=num_labels)
    
    # If the keys in the state_dict are prefixed with 'module.', remove it
    state_dict = checkpoint['model_state_dict']
    new_state_dict = {}
    for key in state_dict:
        new_key = key.replace('module.', '')  # Remove the 'module.' prefix
        new_state_dict[new_key] = state_dict[key]
    
    # Load the state dict into the model
    model.load_state_dict(new_state_dict)
    
    tokenizer = checkpoint['tokenizer']
    label_encoder = checkpoint['label_encoder']
    
    return model, tokenizer, label_encoder

In [3]:
#use this for classifying your text using BERT-based model
def classify_text(path, text, model_class=BertForSequenceClassification, num_labels=10):
    # Load the model, tokenizer, and label encoder
    model, tokenizer, label_encoder = load_model(path, model_class, num_labels)
    
    # Set the model to evaluation mode
    model.eval()
    
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Make sure the inputs are on the correct device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Perform the forward pass and get the logits
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the predicted label (index of the highest logit)
    logits = outputs.logits
    predicted_label_idx = torch.argmax(logits, dim=1).item()
    
    # Convert the label index back to the actual label
    predicted_label = label_encoder.inverse_transform([predicted_label_idx])[0]
    
    return predicted_label


In [10]:
ex1 =  "I''m from Mexico, but I live in Salinas California. The city is a really small and boring. There are some cheap shops and some expensive. There are some buildings. The city has a museum."
ex2 = "In my office there are some desks and chairs. There are some windows. There is a meeting room. There are a few restrooms and there are a lot of peoples."
ex3 = "Hi, My name's Leon. I don't speak english, but I am in learning stage, so I do not have much to say in the newsroom. Good, study the 3rd grade of high school and I intend to exchange. Good Night, bye."
ex4 = "Hi! There will be a great Jenny's party in May on Thursday 6th at 1pm, at No.# of Yellow Street. There will be a lot of foods, chicken and sandwiches, and a lot of drinks, water and wine. We will be in 30 people and I hope you'll come!"
l1 = classify_text('BERT_3e_downsampled.pkl', ex1)
l1

/home/vmadmin/miniforge3/envs/py39/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
A paramet

'Spanish'